## install selenium

In [ ]:
# 한 번만 실행
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

## import

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import json

In [ ]:
# colab에서는 웹브라우저 창이 뜨지 않아 별도 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

## crawling

In [ ]:
driver.get("http://www.exterminal.co.kr/line.asp")

In [ ]:
# iframe(웹페이지 안 웹페이지) 태그로 변경
driver.switch_to.frame(driver.find_element(By.XPATH, '//iframe[@src="./include/kobus_rsv2.asp"]'))

In [ ]:
def get_route():
    route = {}

    # 출발지 선택
    select_dep = Select(driver.find_element(By.ID, "regForm01"))
    options_dep = select_dep.options

    for i in tqdm(range(1, len(options_dep)), desc="crawling..."):
        select_dep.select_by_index(i)
        dep = select_dep.first_selected_option.text
    
        # 도착지 목록 가져오기
        select_arr = Select(driver.find_element(By.ID, "regForm02"))
        options = select_arr.options

        arr = []
        for option in options:
            arr.append(option.text)
    
        route[dep] = arr
    
    return route

In [ ]:
route = get_route()

In [ ]:
keys = []
values = []

for key, value in route.items():
     for v in value:
        keys.append(key)
        values.append(v)

route_df = pd.DataFrame({'dep' : keys, 'arr' : values})

## 노선 정제

In [ ]:
def regular_expression(text):
    pattern = '[가-힣]+'
    result = re.compile(pattern).findall(text)
    if len(result) > 0:
        result = result[0]
    else:
        result = np.nan

    return result

In [ ]:
route_df['dep'] = route_df['dep'].apply(lambda x : regular_expression(x))

* 크롤링시 문제같은데 도착지에 null값이 있음 -> 크롤링 코드를 수정해도 같음
* 함수 적용 후 일단은 null값 제거

In [ ]:
route_df['arr'] = route_df['arr'].apply(lambda x : regular_expression(str(x)))

In [ ]:
route_df = route_df.dropna()

In [ ]:
# save
route_df.to_csv('route.csv', index = False)